# Test Webcam Feed

In [18]:
import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Webcam", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Sample Project with Webcam Feed

In [23]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

model = YOLO('yolov8n.pt')
tracker = DeepSort(max_age=30)
ROI_TOP_LEFT = (200, 100)
ROI_BOTTOM_RIGHT = (400, 300)
person_class_id = 0
confidence_threshold = 0.75
person_count = 0
memory = {}

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.rectangle(frame, ROI_TOP_LEFT, ROI_BOTTOM_RIGHT, (0, 255, 0), 2)
    results = model(frame)
    detections = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        for box, cls, conf in zip(boxes, classes, confs):
            if int(cls) == person_class_id and conf > confidence_threshold:
                x1, y1, x2, y2 = box
                detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        center_x = (x1 + x2) // 2

        # حافظه برای هر فرد
        if track_id not in memory:
            memory[track_id] = {'entered': None, 'exited': None}
            # ورود از سمت راست
            if center_x > ROI_BOTTOM_RIGHT[0]:
                memory[track_id]['entered'] = 'right'
            # ورود از سمت چپ
            elif center_x < ROI_TOP_LEFT[0]:
                memory[track_id]['entered'] = 'left'
        else:
            # خروج از سمت چپ
            if center_x < ROI_TOP_LEFT[0]:
                memory[track_id]['exited'] = 'left'
            # خروج از سمت راست
            elif center_x > ROI_BOTTOM_RIGHT[0]:
                memory[track_id]['exited'] = 'right'

        # منطق شمارش
        if memory[track_id]['entered'] == 'right' and memory[track_id]['exited'] == 'left':
            person_count += 1
            memory[track_id]['entered'] = None  # فقط یکبار بشمارد
        elif memory[track_id]['entered'] == 'left' and memory[track_id]['exited'] == 'right':
            person_count = max(0, person_count - 1)
            memory[track_id]['entered'] = None

        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f"ID:{track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    cv2.putText(frame, f"Persons: {person_count}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.imshow("Webcam", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 1 chair, 1 tv, 58.4ms
Speed: 1.9ms preprocess, 58.4ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 49.4ms
Speed: 1.3ms preprocess, 49.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 1 tv, 43.4ms
Speed: 1.0ms preprocess, 43.4ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 chairs, 1 tv, 31.0ms
Speed: 1.0ms preprocess, 31.0ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 1 tv, 33.2ms
Speed: 1.9ms preprocess, 33.2ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 1 tv, 42.3ms
Speed: 1.6ms preprocess, 42.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 bottle, 2 chairs, 1 tv, 33.0ms
Speed: 1.4ms preprocess, 33.0ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480

## Taking Photos from my own object

In [11]:
cap = cv2.VideoCapture(0)
img_count = 10

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Webcam", frame)
    # اگر کلید 's' را بزنی، عکس ذخیره می‌شود

    key = cv2.waitKey(1)
    if key == ord('s'):
        cv2.imwrite(f"own_images/object_{img_count}.jpg", frame)
        print(f"عکس شماره {img_count} ذخیره شد.")
        img_count += 1
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@303.851] global cap_v4l.cpp:803 requestBuffers VIDEOIO(V4L2:/dev/video0): failed VIDIOC_REQBUFS: errno=19 (No such device)
